In [146]:
import random
import cv2
import numpy as np
import os
from collections import Counter

path = "/home/supercgor/gitfile/data/raw_combine"
total = 700
data_set = {
    # "HDA_3": (40, 40, 3),
    # "HDA_4": (30, 40, 3),
    # "HDA_5": (32, 43, 3),
    # "HDA_6": (38, 30, 3),
    # "HDA_7": (40, 40, 3),
    # "HDA_8": (55, 28, 3),
    "super_structure" : (70, 70, 3)
}

pixel = {
    "HDA_3": (208, 208),
    "HDA_4": (300, 400),
    "HDA_5": (320, 430),
    "HDA_6": (380, 300),
    "HDA_7": (400, 400),
    "HDA_8": (550, 280),
    "super_structure" : (470, 474)
}


In [147]:
out = "/home/supercgor/gitfile/data/lots_exp/super_structure/afm"

In [148]:
keys = list(data_set.keys())
area = [(data_set[key][0] - 25) * (data_set[key][1] - 25) * (pixel[key][0] * pixel[key][1]) for key in keys]
print(keys)
print(area)
keys = random.choices(keys, weights = area, k = total)
keys = Counter(keys)

['super_structure']
[451129500]


In [149]:
print(keys)

Counter({'super_structure': 700})


In [150]:
def rotate_img(img, ang):
    (h, w, d) = img.shape # 讀取圖片大小
    # 第一個參數旋轉中心，第二個參數旋轉角度(-順時針/+逆時針)，第三個參數縮放比例
    M = cv2.getRotationMatrix2D((w//2, h//2), ang, 1.0)
    
    # 第三個參數變化後的圖片大小
    rotate_img = cv2.warpAffine(img, M, (w, h))
    
    return rotate_img

def process_Imgs(imgs: list, org_size: tuple, center: tuple, crop_size: tuple, flip: None | int = None, ang: int = 0):

    cx, cy = center
    crop_X, crop_Y = crop_size
    for _ in range(len(imgs)):
        img = imgs.pop(0)
        img = img[cy:cy + crop_Y, cx: cx+ crop_X]
        img = rotate_img(img, ang = ang)
        H, W, d = img.shape
        img = img[- (pic_y//2) + H//2: pic_y//2 + H//2, - (pic_x//2) + W//2: pic_x//2 + W//2]
        print(img.shape)
        if flip != 2:
            img = cv2.flip(img, flip)
        img = cv2.resize(img, (128,128))
        imgs.append(img)
    
    return imgs

In [151]:
for key in keys:
    imgs = []
    nums = keys[key]
    for i in range(len(os.listdir(f"{path}/afm/{key}"))):
        img = cv2.imread(f"{path}/afm/{key}/{i}.png")
        if key == "super_structure":
            img = cv2.resize(img, (472, 472))
        imgs.append(img)
    X,Y = pixel[key]
    x, y, z = data_set[key]
    pic_x = int(X / x * 25)
    pic_y = int(Y / y * 25)
    min_ang = 0
    max_ang = np.pi/2
    if X/ np.sqrt(pic_x**2 + pic_y**2) <= 1:
        max_ang = np.arcsin(X/np.sqrt(pic_x**2 + pic_y**2)) - np.arcsin(pic_x/np.sqrt(pic_x**2 + pic_y**2))
    if Y/ np.sqrt(pic_x**2 + pic_y**2) <= 1:
        min_ang = np.arccos(Y/np.sqrt(pic_x**2 + pic_y**2)) + np.arcsin(pic_x/np.sqrt(pic_x**2 + pic_y**2))
    min_deg = np.ceil(min_ang * 180 / np.pi)
    max_deg = np.floor(max_ang * 180 / np.pi)
    #print(min_deg, max_deg)
    while nums > 0:
        pics = imgs.copy()
        ang_deg = random.randint(min_deg, max_deg)
        ang_rad = ang_deg / 180 * np.pi
        crop_X, crop_Y = np.cos(ang_rad) * pic_x + np.sin(ang_rad) * pic_y, np.sin(ang_rad) * pic_x + np.cos(ang_rad) * pic_y
        crop_X, crop_Y = int(np.floor(crop_X)), int(np.floor(crop_Y))
        cx,cy = random.randint(0,X - crop_X) ,random.randint(0, Y - crop_Y)
        flip = random.choice([2, -1, 0, 1])
        ang_deg = random.choice([-1,1]) * ang_deg
        pics = process_Imgs(pics, (X,Y), (cx,cy), (crop_X, crop_Y), flip = flip, ang = ang_deg)
        # pics = np.concatenate(pics, axis = 1)
        # cv2.imshow(f"{cx},{cy},{ang_deg}", pics)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # if pics[0].shape != (128,128, 3):
        #     break
        if not os.path.exists(f"{out}/{key}-{cx}_{cy}_{ang_deg}_{flip}"):
            print(f"key: {key} ; cx,cy: {cx},{cy} ; ang: {ang_deg} ; flip: {flip}")
            os.mkdir(f"{out}/{key}-{cx}_{cy}_{ang_deg}_{flip}")
            for i,pic in enumerate(pics):
                cv2.imwrite(f"{out}/{key}-{cx}_{cy}_{ang_deg}_{flip}/{i}.png", pic)
            nums -= 1

(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
key: super_structure ; cx,cy: 50,36 ; ang: -29 ; flip: 2
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
key: super_structure ; cx,cy: 227,193 ; ang: 44 ; flip: 2
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
key: super_structure ; cx,cy: 186,218 ; ang: -53 ; flip: 1
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
key: super_structure ; cx,cy: 235,3 ; ang:

In [152]:
print(f"key: {key} ; cx,cy: {cx},{cy} ; ang: {ang_deg} ; flip: {flip}")

process_Imgs(imgs, (X,Y), (cx,cy), (crop_X, crop_Y), flip = flip, ang = ang_deg)

key: super_structure ; cx,cy: 201,5 ; ang: -46 ; flip: 1
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)
(168, 166, 3)


[array([[[255, 157,  69],
         [255, 166,  88],
         [255, 173, 101],
         ...,
         [238, 113,   0],
         [230, 109,   0],
         [210, 100,   0]],
 
        [[255, 146,  48],
         [255, 153,  61],
         [255, 161,  76],
         ...,
         [237, 112,   0],
         [231, 110,   0],
         [224, 107,   0]],
 
        [[253, 133,  24],
         [255, 141,  37],
         [255, 144,  45],
         ...,
         [238, 113,   0],
         [232, 110,   0],
         [225, 107,   0]],
 
        ...,
 
        [[229, 109,   0],
         [229, 108,   0],
         [227, 108,   0],
         ...,
         [255, 135,  25],
         [255, 128,  14],
         [253, 121,   2]],
 
        [[236, 112,   0],
         [234, 111,   0],
         [232, 110,   0],
         ...,
         [255, 133,  24],
         [255, 126,  11],
         [251, 121,   4]],
 
        [[242, 115,   0],
         [240, 114,   0],
         [236, 112,   0],
         ...,
         [255, 128,  15],
  

In [153]:
for pic in pics:
    print(pic.shape)
new = np.concatenate(pics, axis = 1)
cv2.imshow(f"{cx},{cy},{ang_deg}", new)
cv2.waitKey(0)
cv2.destroyAllWindows()

(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)
(128, 128, 3)


In [154]:
-pic_y//2 + H//2

NameError: name 'H' is not defined

In [ ]:
- (155.5 //2)